# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

city_weather_df = "../output_data/cities.csv"
city_vaction_df = pd.read_csv(city_weather_df)
city_vaction_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Opuwo,-18.0607,13.8400,71.58,35,89,4.23,NaN,1666664760
1,Hermanus,-34.4187,19.2345,61.81,70,100,10.09,ZA,1666665033
2,Rikitea,-23.1203,-134.9692,73.35,81,2,15.32,PF,1666665024
3,Tabou,4.4230,-7.3528,76.35,92,38,7.29,CI,1666664613
4,Thompson,55.7435,-97.8558,30.36,93,100,18.41,CA,1666664633
...,...,...,...,...,...,...,...,...,...
558,Imeni Poliny Osipenko,52.4201,136.4853,40.48,41,61,8.10,RU,1666665392
559,Yakima,46.4665,-120.6678,53.53,58,0,21.85,US,1666665393
560,Beringovskiy,63.0500,179.3167,30.11,82,100,7.31,RU,1666665393
561,Arrondissement d'Arcachon,44.5875,-1.0519,53.92,87,0,0.00,FR,1666665393


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
city_vaction_df.dtypes

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [5]:
#gmaps.configure(api_key=g_key)
#gmaps.figure()

In [6]:
locations = city_vaction_df[["Lat", "Lng"]]
humidity = city_vaction_df["Humidity"]

fig=gmaps.figure(center=(50.0, -35.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
vacation_df = city_vaction_df[(city_vaction_df["Max Temp"] >70) & (city_vaction_df["Max Temp"] <80) \
                            & (city_vaction_df["Wind Speed"] <10) & (city_vaction_df["Cloudiness"] ==0)]\
                              .dropna()
vacation_df                           

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,Salalah,17.0151,54.0924,75.29,73,0,4.61,OM,1666665210
67,Mānsa,29.9833,75.3833,73.22,30,0,2.89,IN,1666665216
112,Lar,26.2039,83.9717,70.52,66,0,6.02,IN,1666664716
137,Gwadar,25.1216,62.3254,79.41,69,0,8.93,PK,1666665236
278,Alibag,18.6411,72.8792,77.07,62,0,3.67,IN,1666665283
339,Pinos Puente,37.2511,-3.7497,73.29,34,0,0.00,ES,1666665303
427,Abu Dhabi,24.4667,54.3667,78.82,73,0,2.30,AE,1666665340
548,Umluj,25.0213,37.2685,76.86,57,0,9.75,SA,1666665387


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = vacation_df.loc[:,["City", "Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""

#display
hotel_df

,City,Country,Lat,Lng,Hotel Name
44,Salalah,OM,17.0151,54.0924,
67,Mānsa,IN,29.9833,75.3833,
112,Lar,IN,26.2039,83.9717,
137,Gwadar,PK,25.1216,62.3254,
278,Alibag,IN,18.6411,72.8792,
339,Pinos Puente,ES,37.2511,-3.7497,
427,Abu Dhabi,AE,24.4667,54.3667,
548,Umluj,SA,25.0213,37.2685,


In [9]:
import json
import pprint

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


for index, row in hotel_df.iterrows(): 
   # print(row)
    #break 
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    pprint.pprint(response)
    
    # extract results
    results = response['results']
    
   #print(json.dumps(response, indent=4, sort_keys=True))
#     break

# save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except (KeyError, IndexError):
        print("No hotel")

Retrieving Results for Index 44: Salalah.
{'html_attributions': [],
 'next_page_token': 'AcYSjRiWwNcQMGP_nkNCVF6sOQNNnDpLbsPKyyoUUTVMHaIyuOfdaqBfhy2CxnG3tzGlMisSES9LCa4Bpi9QqEOQidMNcl38tIlUn2-N7I3lnTKdFtQN3JlR-dus5KjaeKjNDQNFm2V4EFabsTq8uVWFJfsnfpSYR7XKpKmEStt8g8_L_lTTzT9m7vjrgYx1BalID-KWNvGKj_SUNAfH9I86B4zDAqpeE2XnMthrDvpuZthZYZVnbYG2MCVVcsDPYdAcEFCukHbsEqPCPyIrooCyRbSkh_ToMvjLODDp2hut9slLYCK53HM4ZNzwCvNhjUfMbbF0GQbhn6SCBSTcJ825NwYGEDNDTI5xQZNBiu7K-Oad0ovgk8n6e7ke8E-zJjJIy1UZC47g6T9RzgwEebkuNxmp-SD1ivJ2xhNdFcRjQur-9lFY_0ZSjBO6PnXuZZU',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 17.0105468,
                                        'lng': 54.07818839999999},
                           'viewport': {'northeast': {'lat': 17.01197842989272,
                                                      'lng': 54.07951227989273},
                                        'southwest': {'lat': 17.00927877010728,
                                          

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 29.9732367,
                                        'lng': 75.38198489999999},
                           'viewport': {'northeast': {'lat': 29.97462587989273,
                                                      'lng': 75.38333382989272},
                                        'southwest': {'lat': 29.97192622010729,
                                                      'lng': 75.38063417010729}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Hotel 4 Seasons',
              'photos': [{'height': 2656,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/maps/

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 26.1939383, 'lng': 83.9761401},
                           'viewport': {'northeast': {'lat': 26.19525297989272,
                                                      'lng': 83.97747697989273},
                                        'southwest': {'lat': 26.19255332010728,
                                                      'lng': 83.97477732010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Rajkishor Hotel',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 1040,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/m

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 25.1032914,
                                        'lng': 62.32863299999999},
                           'viewport': {'northeast': {'lat': 25.10447642989272,
                                                      'lng': 62.32994732989271},
                                        'southwest': {'lat': 25.10177677010728,
                                                      'lng': 62.32724767010727}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'Zaver Pearl Continental Hotel',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 1068,
                          'html_attributions': ['<a '
                   

{'html_attributions': [],
 'next_page_token': 'AcYSjRiCfEv7JkRmfn9nHhj62N82LzRGom-SZdlqE2d5jvO9eW8TIYnjGxO4SskFTir_CGpMNI84PBad-RTryGP2tkVO1QSgmAOZ_rXaFSKW3ESiIyFbpZjL1wdgH37uzRgTKWHeSf-cn1yYiSvPMQQjGahA-gaFra_4G5esunoFGYK5px0U7iOLFLE6KPmE2iURfdc_gY-Ont0GlYxD8LgmGjtMV0bgHvJqPMlSDQNfUE106aJFIlD3w_FfUZ_aikWCpg_VGpgxs6kqADtbU9frSYTMQZak83gJCg3H4g9481s0IjYxA-upSzMrWLhb0PYVD-Unmp7yKFLdkXhBw3nt7Xr2VCKLCUWytcpKYIWKc8UcTlato9EgOPAXqA9Jp1U6WJVL7983tS7NcTsofK4tM_CYkaB-76MIWBrbiv5Fw2FzIdRTZ24iYlqgIQ7A6EY',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 18.6650763, 'lng': 72.8890831},
                           'viewport': {'northeast': {'lat': 18.66600832989272,
                                                      'lng': 72.88988079999999},
                                        'southwest': {'lat': 18.66330867010728,
                                                      'lng': 72.88668999999999}}},
              'icon': 'https://maps.gstatic.com/

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 37.2484261, 'lng': -3.7465202},
                           'viewport': {'northeast': {'lat': 37.24971237989273,
                                                      'lng': -3.745158820107278},
                                        'southwest': {'lat': 37.24701272010729,
                                                      'lng': -3.747858479892721}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
              'icon_background_color': '#FF9E67',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
              'name': 'Hotel Restaurante Montserrat',
              'opening_hours': {'open_now': False},
              'photos': [{'height': 3000,
                          'html_attributions': ['<a '
                                                'href="h

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 25.0325904, 'lng': 37.2570057},
                           'viewport': {'northeast': {'lat': 25.03390552989272,
                                                      'lng': 37.25838357989272},
                                        'southwest': {'lat': 25.03120587010728,
                                                      'lng': 37.25568392010728}}},
              'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
              'icon_background_color': '#909CE1',
              'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
              'name': 'HP Red Sea Hotel',
              'opening_hours': {'open_now': True},
              'photos': [{'height': 4032,
                          'html_attributions': ['<a '
                                                'href="https://maps.google.com/

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
hotel_info
#Turns to html code

['\n<dl>\n<dt>Name</dt><dd>Darbat Hotel</dd>\n<dt>City</dt><dd>Salalah</dd>\n<dt>Country</dt><dd>OM</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel 4 Seasons</dd>\n<dt>City</dt><dd>Mānsa</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Rajkishor Hotel</dd>\n<dt>City</dt><dd>Lar</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Zaver Pearl Continental Hotel</dd>\n<dt>City</dt><dd>Gwadar</dd>\n<dt>Country</dt><dd>PK</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Radisson Blu Resort & Spa - Alibaug, India</dd>\n<dt>City</dt><dd>Alibag</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Restaurante Montserrat</dd>\n<dt>City</dt><dd>Pinos Puente</dd>\n<dt>Country</dt><dd>ES</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Jannah Burj Al Sarab Hotel</dd>\n<dt>City</dt><dd>Abu Dhabi</dd>\n<dt>Country</dt><dd>AE</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>HP Red Sea Hotel</dd>\n<dt>City</dt><dd>Umluj</dd>\n<dt>Country</dt><dd>SA</dd>

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)


# Display figure
fig




Figure(layout=FigureLayout(height='420px'))